In [21]:
# import libraries needed for the project

#data handling tools
import pandas as pd
import numpy as np

#data visualization tools
import matplotlib.pyplot as plt
import seaborn as sns

#machine learning tools
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import TextVectorization, Embedding
from sklearn.model_selection import train_test_split

import random


In [2]:
# import the training and testing data
train_df = pd.read_csv('./Data/train.csv')
test_df = pd.read_csv('./Data/test.csv')

In [3]:
# read the head of the dataset to become familiar with the data
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


since we're not working with a sequential data, we might want to shuffle to prevent the models from learning in some order

In [4]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
# looking at the test set
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
# get 5 random tweets and their classes
random_index = random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}', '(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n{text}\n')
  print('---')

Target: 1 (real disaster)
Text:
@CAgov If 90BLKs&amp;8WHTs colluded 2 take WHT F @USAgov AUTH Hostage&amp;2 make her look BLK w/Bioterrorism&amp;use her lgl/org IDis ID still hers?@VP

---
Target: 0 (not real disaster)
Text:
wreck? wreck wreck wreck wreck wreck wreck wreck wreck wreck wreck wreck wreck?

---
Target: 0 (not real disaster)
Text:
failure is a misfortunebut regret is a catastrophe

---
Target: 1 (real disaster)
Text:
@Hurricane_Dame ???????? I don't have them they out here

---
Target: 1 (real disaster)
Text:
#USGS M 1.2 - 23km S of Twentynine Palms California: Time2015-08-05 23:54:09 UTC2015-08-05 16:54:09 -07:0... http://t.co/kF0QYBKZOL #SM

---


In [7]:
# Let's split the data into training and validation dataset
X_train, X_val, y_train, y_val = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                    train_df_shuffled['target'].to_numpy(),
                                                    test_size=0.1, random_state=42)

In [8]:
# check for sizes of the split
len(X_train), len(X_val), len(y_train), len(y_val)

(6851, 762, 6851, 762)

In [9]:
# visualise the training data
X_train[:11], y_val[0:11]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt',
        'Lava Dragon Breeder! I j

## Text Preprocessing

In [18]:
# check for the average number of tokens in each tweet
avg_token = round(sum([len(i.split()) for i in train_df_shuffled['text']])/len(train_df_shuffled['text']))

15

In [19]:
# Hyperparameters
max_vocab_length = 10000
max_length = avg_token

# create an instance of the TextVectorization Layer
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_mode = 'int',
    output_sequence_length = max_length
)

In [20]:
# adapt the tokenizer to the training samples
text_vectorizer.adapt(X_train)

In [23]:
# Get an embedding layer to accurately find the relationship among the tokens
embedding = Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,
)

In [24]:
# get the embedding of a random tweet
random_tweet = random.choice(X_train)
print(f'Original tweet:\n{random_tweet}\
        \n\nEmbedded version:')
embedding(text_vectorizer([random_tweet]))

Original tweet:
Student electrocuted to death in school campus http://t.co/ryah8Fni5Q        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0414767 , -0.03910216, -0.03668414, ..., -0.04813225,
         -0.01508291,  0.02133857],
        [ 0.02991715, -0.01586054, -0.01442079, ...,  0.03518431,
          0.01910175, -0.04345685],
        [ 0.04974233,  0.04207441, -0.01998394, ..., -0.01676161,
         -0.00855368,  0.00716491],
        ...,
        [ 0.04800791, -0.00635643, -0.04636264, ...,  0.01511383,
         -0.0326084 ,  0.02161643],
        [ 0.04800791, -0.00635643, -0.04636264, ...,  0.01511383,
         -0.0326084 ,  0.02161643],
        [ 0.04800791, -0.00635643, -0.04636264, ...,  0.01511383,
         -0.0326084 ,  0.02161643]]], dtype=float32)>